In [ ]:
# import requests
# from gtts import gTTS
# import speech_recognition as sr
# import os
# import ollama
# import time
# import re
# import threading

# api_key = 'c0826860be16ee211a9e15b22d059034'

# # Global flag to control the beep loop
# beep_playing = threading.Event()

# def play_beep():
#     """Plays beep sound repeatedly until stopped."""
#     while not beep_playing.is_set():
#         os.system(f'afplay "beep.mp3"')  # Adjust the command based on your OS
#         time.sleep(1)  # Adjust as necessary for the length of your beep sound

# def speak(text):
#     """Converts text to speech and plays it."""
#     if text.strip():  # Check if the text is not empty or whitespace
#         text_for_speaking = re.sub(r'[.!?]', '', text)
#         tts = gTTS(text=text_for_speaking, lang='en')
#         tts.save('output.mp3')
#         os.system(f'afplay output.mp3')  # Adjust command based on your OS
#     else:
#         print("No text to speak.")

# def listen():
#     """Listens for speech and converts it to text, then prints what was heard."""
#     r = sr.Recognizer()
#     with sr.Microphone() as source:
#         print("Listening...")
#         speak("Listening...")
#         audio = r.listen(source)
#     try:
#         user_input = r.recognize_google(audio)
#         # This line prints what the speech recognition library recognized.
#         print(f"The system heard: '{user_input}'")
#         return user_input
#     except sr.UnknownValueError:
#         print("Sorry, I could not understand your speech.")
#         return ""
#     except sr.RequestError:
#         print("Sorry, I am unable to access the Google Speech Recognition service.")
#         return ""

# def chat_with_ollama():
#     """Manages the chat conversation with Ollama."""
#     print("Starting chat with Ollama. Type 'quit' to exit.")
#     messages = []

#     # Add a system prompt before starting the chat
#     system_prompt = input("Enter initial system prompt for Ollama: ")
#     if system_prompt.strip():
#         # This assumes that the system prompt can be directly appended to messages if needed
#         messages.append({'role': 'system', 'content': system_prompt})

#     while True:
#         user_input = listen()
#         if user_input.lower() == 'quit':
#             break

#         if user_input.strip():
#             messages.append({'role': 'user', 'content': user_input})
            
#             # Start playing beep sound in a loop on a separate thread
#             beep_playing.clear()
#             beep_thread = threading.Thread(target=play_beep)
#             beep_thread.start()

#             stream = ollama.chat(model='mistral', messages=messages, stream=True)
#             complete_response = ""  # Accumulate the complete response for speaking
#             for chunk in stream:
#                 response = chunk['message']['content']
#                 complete_response += " " + response
#                 words = response.split()
                
#                 for word in words:
#                     print(word, end=' ', flush=True)
#                     time.sleep(0.2)

#             # Stop the beep sound and wait for the thread to finish
#             beep_playing.set()
#             beep_thread.join()

#             # Speak the complete response
#             if complete_response.strip():
#                 speak(complete_response.strip())
#         else:
#             print("Message is empty, please enter some text.")

# if __name__ == "__main__":
#     chat_with_ollama()


In [2]:
import requests
from gtts import gTTS
import speech_recognition as sr
import os
import ollama
import time
import re
import threading
import csv

api_key = 'c0826860be16ee211a9e15b22d059034'

# Global flag to control the beep loop
beep_playing = threading.Event()

def play_beep():
    """Plays beep sound repeatedly until stopped."""
    while not beep_playing.is_set():
        os.system(f'afplay "beeping.mp3"')  # Use 'start beep.mp3' on Windows or 'mpg123 beep.mp3' on Linux
        time.sleep(1)  # Adjust as necessary for the length of your beep sound

def speak(text):
    """Converts text to speech and plays it."""
    if text.strip():  # Check if the text is not empty or whitespace
        text_for_speaking = re.sub(r'[.!?]', '', text)
        tts = gTTS(text=text_for_speaking, lang='en')
        tts.save('output.mp3')
        os.system(f'afplay output.mp3')  # Adjust command based on your OS
    else:
        print("No text to speak.")

def listen():
    """Listens for speech and converts it to text."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        speak("Listening...")
        print("\nListening...")
        audio = r.listen(source)
    try:
        user_input = r.recognize_google(audio)
        # This line prints what the speech recognition library recognized.
        print(f"The system heard: '{user_input}'")
        speak(f"The system heard: '{user_input}'")
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand your speech.")
        return ""
    except sr.RequestError:
        print("Sorry, I am unable to access the Google Speech Recognition service.")
        return ""

def read_mobility_canes(filename):
    """Reads the content of a text file into a list."""
    with open(filename, 'r') as file:
        content = file.readlines()
    return [line.strip() for line in content]

def read_tips_csv(filename):
    """Reads a CSV file into a list of dictionaries."""
    with open(filename, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return [row for row in reader]
def chat_with_ollama(mobility_canes_data, tips_data):
    """Manages the chat conversation with Ollama."""
    print("Starting chat with Ollama. Type 'quit' to exit.")
    messages = []

    # Add a system prompt before starting the chat
    system_prompt = input("Enter initial system prompt for Ollama: ")
    system_prompt.append("Ask the person to tell their height, cane technique and environments they travese. You are an assistant to help them with their mobility. Give really short and concise tips.")
    if system_prompt.strip():
        # Ensure that system prompts are non-empty before appending
        messages.append({'role': 'system', 'content': system_prompt})
        if mobility_canes_data:
            mobility_content = '\n'.join(mobility_canes_data)
            if mobility_content.strip():
                messages.append({'role': 'system', 'content': mobility_content})
        if tips_data:
            # Assume 'tip' is a key in the dictionaries contained within tips_data
            tips_content = '\n'.join(tip['tip'] for tip in tips_data if 'tip' in tip and tip['tip'].strip())
            if tips_content.strip():
                messages.append({'role': 'system', 'content': tips_content})
        
        user_input = input("Enter the city you wish to traverse: ")
        try:
            weather_data = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={user_input}&units=metric&APPID={api_key}") # 
            weather_json = weather_data.json()

            if weather_json['cod'] == 200:
                temperature_message =  f"You will experience {weather_json['weather'][0]['description']} in {user_input} today." # f"The temperature in {user_input} is {weather_json['main']['temp']}°C."
                speak(temperature_message)
                if temperature_message.strip():
                    messages.append({'role': 'system', 'content': temperature_message})
        except requests.exceptions.RequestException as e:
            print(f"Error fetching weather data: {str(e)}")

    while True:
        user_input = listen()
        if user_input.lower() == 'quit':
            break

        if user_input.strip():
            messages.append({'role': 'user', 'content': user_input})
            
            # Start playing beep sound in a loop on a separate thread
            beep_playing.clear()
            beep_thread = threading.Thread(target=play_beep)
            beep_thread.start()

            try:
                stream = ollama.chat(model='mistral', messages=messages, stream=True)
                complete_response = ""  # Accumulate the complete response for speaking
                for chunk in stream:
                    response = chunk['message']['content']
                    complete_response += " " + response
                    words = response.split()
                    
                    for word in words:
                        print(word, end=' ', flush=True)
                        time.sleep(0.2)
            except Exception as e:
                print(f"Error during Ollama chat: {str(e)}")
                break

            # Stop the beep sound and wait for the thread to finish
            beep_playing.set()
            beep_thread.join()

            # Speak the complete response
            if complete_response.strip():
                speak(complete_response.strip())
        else:
            print("Message is empty, please enter some text.")


if __name__ == "__main__":
    # Reading file names from system input
    mobility_canes_file = '/Users/ssk11/Downloads/llama_work/.venv/mobility-canes.txt'
    tips_csv_file = '/Users/ssk11/Downloads/llama_work/.venv/tips.csv'
    
    # Reading the contents of the files
    mobility_canes_data = read_mobility_canes(mobility_canes_file)
    tips_data = read_tips_csv(tips_csv_file)
    
    # Now you can use mobility_canes_data and tips_data in your application
    chat_with_ollama(mobility_canes_data, tips_data)
